# **Bee Cycle - SQL Queries for Data Cleaning**

---



This file contains some of the queries I used in order to check and clean the data. <br>

## **1) Customer Table**

Check length of rows and columns - 1360 rows and 7 columns. <br>
Check all variables are correct data type - they are.

**a) Preview table**

In [18]:
query = '''
SELECT *
FROM `beecycle-413111.BeeCycle.dim_customer`
LIMIT 3;
'''

df = gcpdf(query)
df

,customer_id,geography_id,customer_name,birthdate,maritalstatus,gender,datefirstpurchase
0,13591,278,Latasha Alonso,1973-05-25,M,F,2017-07-05
1,13264,121,Melanie Sanchez,1976-05-23,M,F,2017-07-06
2,13258,133,Anne Dominguez,1977-03-03,M,F,2017-07-08


**b) Check for any null values**

In [19]:
query = '''
SELECT *
FROM `beecycle-413111.BeeCycle.dim_customer`
WHERE customer_id IS NULL OR geography_id IS NULL OR customer_name IS NULL OR birthdate IS NULL OR maritalstatus IS NULL OR gender IS NULL OR datefirstpurchase IS NULL;
'''

df = gcpdf(query)
df

,customer_id,geography_id,customer_name,birthdate,maritalstatus,gender,datefirstpurchase
0,99997,37,Siska Budi,1996-10-29,S,F,NaT
1,99998,26,Diana Saraswati,1997-09-26,S,F,NaT
2,99999,37,Aisyah Soebandono,1996-03-27,S,F,NaT


There are 3 null values in the '*datefirstpurchase'* column. However, this only suggests that these customers haven't yet made a first purchase, and so we will not remove them from the dataset.

**c) Check for any duplicate values**

In [20]:
query = '''
SELECT COUNT(*)
FROM `beecycle-413111.BeeCycle.dim_customer`
GROUP BY customer_id, geography_id, customer_name, birthdate, maritalstatus, gender, datefirstpurchase
HAVING COUNT(*) > 1;
'''

df = gcpdf(query)
df

,f0_


There are no duplicates.

**d) Check how many unique customer id's there are**

In [21]:
query = '''
SELECT COUNT(DISTINCT customer_id) AS customer_count
FROM `beecycle-413111.BeeCycle.dim_customer`;
'''

df = gcpdf(query)
df

,customer_count
0,1360


There are 1360 unique customers. Now let's see whether there are any customers with the same name.

In [22]:
query = '''
SELECT *
FROM (
  SELECT
    customer_name,
    COUNT(*) OVER (PARTITION BY customer_name) AS name_count,
    `beecycle-413111.BeeCycle.dim_customer`.*
  FROM
    `beecycle-413111.BeeCycle.dim_customer`
) AS CustomerDuplicates
WHERE name_count > 1;
'''

df = gcpdf(query)
df

,customer_name,name_count,customer_id,geography_id,customer_name_1,birthdate,maritalstatus,gender,datefirstpurchase
0,Mohamed Pal,2,12698,13,Mohamed Pal,1990-05-14,M,M,2019-09-14
1,Mohamed Pal,2,28763,302,Mohamed Pal,1994-07-23,S,F,2017-11-16


There are two seperate people named Mohamed Pal.

**e) Check how many unique geography id's there are**

In [23]:
query = '''
SELECT COUNT(DISTINCT geography_id) AS geography_count
FROM `beecycle-413111.BeeCycle.dim_customer`;
'''

df = gcpdf(query)
df

,geography_count
0,279


The 1360 customers are spread out between 279 different places around the world.

**f) Check how many different genders and marital statuses there are**

In [24]:
query = '''
SELECT DISTINCT maritalstatus, gender
FROM `beecycle-413111.BeeCycle.dim_customer`;
'''

df = gcpdf(query)
df

,maritalstatus,gender
0,M,F
1,M,M
2,S,F
3,S,M


Customers can either be married (M) or single (S) and male (M) or female (F).

**g) Check for the minimum and maximum birthdate and date of first purchase**


In [25]:
query = '''
SELECT
  MIN(birthdate) AS min_birthdate,
  MAX(birthdate) AS max_birthdate,
  MIN(datefirstpurchase) AS min_datefirstpurchase,
  MAX(datefirstpurchase) AS max_datefirstpurchase
FROM `beecycle-413111.BeeCycle.dim_customer`;
'''

df = gcpdf(query)
df

,min_birthdate,max_birthdate,min_datefirstpurchase,max_datefirstpurchase
0,1946-07-08,2000-11-06,2017-07-01,2021-01-02


There does not appear to be any impossible values here.

<br>

---



## **2) Geography Table**

Check length of rows and columns - 654 rows and 7 columns. <br>
Check all variables are correct data type - they are.

**a) Preview table**

In [26]:
query = '''
SELECT *
FROM `beecycle-413111.BeeCycle.dim_geography`
LIMIT 3;
'''

df = gcpdf(query)
df

,geography_id,city,state_province_code,state_province_name,country_region_code,english_country_region_name,postal_code
0,2,Coffs Harbour,NSW,New South Wales,AU,Australia,2450
1,3,Darlinghurst,NSW,New South Wales,AU,Australia,2010
2,4,Goulburn,NSW,New South Wales,AU,Australia,2580


**b) Check for any null values**

In [27]:
query = '''
SELECT *
FROM `beecycle-413111.BeeCycle.dim_geography`
WHERE geography_id IS NULL OR city IS NULL OR state_province_code IS NULL OR state_province_name IS NULL OR country_region_code IS NULL OR english_country_region_name IS NULL OR postal_code IS NULL;
'''

df = gcpdf(query)
df

,geography_id,city,state_province_code,state_province_name,country_region_code,english_country_region_name,postal_code


There are no null values.

**c) Check for any duplicates**

In [28]:
query = '''
SELECT COUNT(*)
FROM `beecycle-413111.BeeCycle.dim_geography`
GROUP BY geography_id, city, state_province_code, state_province_name, country_region_code, english_country_region_name, postal_code
HAVING COUNT(*) > 1;
'''

df = gcpdf(query)
df

,f0_


There are no duplicates.

**d) Check how many unique values exist for each variable**

In [29]:
query = '''
SELECT
  COUNT(DISTINCT geography_id) AS geography_count,
  COUNT(DISTINCT city) AS city_count,
  COUNT(DISTINCT state_province_code) AS province_code_count,
  COUNT(DISTINCT state_province_name) AS province_name_count,
  COUNT(DISTINCT postal_code) AS post_code_count
FROM `beecycle-413111.BeeCycle.dim_geography`;
'''
df = gcpdf(query)
df

,geography_count,city_count,province_code_count,province_name_count,post_code_count
0,654,561,71,71,634


There are 654 unique possible geography id's (locations around the world) within 561 cities and 634 post codes.

In [30]:
query = '''
SELECT DISTINCT country_region_code, english_country_region_name,
FROM `beecycle-413111.BeeCycle.dim_geography`;
'''

df = gcpdf(query)
df

,country_region_code,english_country_region_name
0,AU,Australia
1,CA,Canada
2,DE,Germany
3,FR,France
4,GB,United Kingdom
5,US,United States


There are 6 unique region codes corresponding to 6 countries.

<br>

---



## **3) Product Table**

Check length of rows and columns - 397 rows and 9 columns. <br>
Check all variables are correct data type - they are.

**a) Preview table**

In [31]:
query = '''
SELECT *
FROM `beecycle-413111.BeeCycle.dim_product`
LIMIT 3;
'''

df = gcpdf(query)
df

,product_id,product_name,model_name,color,size_range,cost,normal_price,sub_category,category
0,604,"Road-750 Black, 44",Road-750,Black,42-46 CM,240.6,378.0,Road Bikes,Bikes
1,605,"Road-750 Black, 48",Road-750,Black,48-52 CM,240.6,378.0,Road Bikes,Bikes
2,606,"Road-750 Black, 52",Road-750,Black,48-52 CM,240.6,378.0,Road Bikes,Bikes


**b) Check for any null values**

In [32]:
query = '''
SELECT *
FROM `beecycle-413111.BeeCycle.dim_product`
WHERE product_id IS NULL OR product_name IS NULL OR model_name IS NULL OR color IS NULL OR size_range IS NULL OR cost IS NULL OR normal_price IS NULL OR sub_category IS NULL OR category IS NULL;
'''

df = gcpdf(query)
df

,product_id,product_name,model_name,color,size_range,cost,normal_price,sub_category,category
0,224,AWC Logo Cap,Cycling Cap,Multi,None,3.7,6.1,Caps,Clothing
1,223,AWC Logo Cap,Cycling Cap,Multi,None,4.0,6.1,Caps,Clothing
2,225,AWC Logo Cap,Cycling Cap,Multi,None,4.8,6.3,Caps,Clothing
3,391,LL Fork,LL Fork,None,None,46.1,103.8,Forks,Components
4,392,ML Fork,ML Fork,None,None,54.5,122.8,Forks,Components
...,...,...,...,...,...,...,...,...,...
93,540,HL Road Tire,HL Road Tire,None,None,8.5,22.8,Tires and Tubes,Accessories
94,537,HL Mountain Tire,HL Mountain Tire,None,None,9.2,24.5,Tires and Tubes,Accessories
95,477,Water Bottle - 30 oz.,Water Bottle,None,None,1.3,3.5,Bottles and Cages,Accessories
96,479,Road Bottle Cage,Road Bottle Cage,None,None,2.4,6.3,Bottles and Cages,Accessories


There are 98 rows where at least one column contains a null value. From inspection, it appears the nulls are just in the '*color*' and '*size_range*' columns. Let's check:

In [33]:
query = '''
SELECT COUNT(*)
FROM `beecycle-413111.BeeCycle.dim_product`
WHERE color IS NULL OR size_range IS NULL;
'''

df = gcpdf(query)
df

,f0_
0,98


We were right. This is important to remember for our analyses as these columns may not be so useful to us with so much missing information.

**c) Check for any duplicates**

In [34]:
query = '''
SELECT COUNT(*)
FROM `beecycle-413111.BeeCycle.dim_product`
GROUP BY product_id, product_name, model_name, color, size_range, cost, normal_price, sub_category, category
HAVING COUNT(*) > 1;
'''

df = gcpdf(query)
df

,f0_


There are none.

**d) Check how many unique values exist for certain variables**

In [35]:
query = '''
SELECT
  COUNT(DISTINCT product_id) AS id_count,
  COUNT(DISTINCT product_name) AS product_name_count,
  COUNT(DISTINCT model_name) AS model_count,
  COUNT(DISTINCT sub_category) AS sub_category_count,
  COUNT(DISTINCT category) AS category_count
FROM `beecycle-413111.BeeCycle.dim_product`;
'''

df = gcpdf(query)
df

,id_count,product_name_count,model_count,sub_category_count,category_count
0,397,295,119,37,4


There are 397 unique product id's, 295 product names, 119 model names, 37 sub categories, and 4 categories.


In [36]:
query = '''
SELECT
  color,
  COUNT(color) AS color_count
FROM `beecycle-413111.BeeCycle.dim_product`
GROUP BY color
ORDER BY color_count DESC;
'''

df = gcpdf(query)
df

,color,color_count
0,Black,129
1,Red,63
2,Yellow,46
3,Silver,45
4,Blue,28
5,Multi,18
6,Silver/Black,7
7,White,4
8,Grey,1
9,None,0


These are all the possible colours that a product could be and how many times they appear.

In [37]:
query = '''
SELECT
  size_range,
  COUNT(size_range) AS size_range_count
FROM `beecycle-413111.BeeCycle.dim_product`
GROUP BY size_range
ORDER BY size_range_count DESC;
'''

df = gcpdf(query)
df

,size_range,size_range_count
0,42-46 CM,80
1,48-52 CM,70
2,54-58 CM,35
3,60-62 CM,35
4,38-40 CM,33
5,L,14
6,M,14
7,S,12
8,XL,5
9,70,1


These are all the different size ranges a product could be. There seems to be inconsistent modes of measurement, which makes it difficult to compare across categories.

**e) Check for the minimum, maximum and average cost and price**

In [38]:
query = '''
SELECT
  MIN(cost) AS min_cost,
  MAX(cost) AS max_cost,
  AVG(cost) AS average_cost,
  MIN(normal_price) AS min_price,
  MAX(normal_price) AS max_price,
  AVG(normal_price) AS average_price
FROM `beecycle-413111.BeeCycle.dim_product`;
'''

df = gcpdf(query)
df

,min_cost,max_cost,average_cost,min_price,max_price,average_price
0,0.6,1519.9,302.460453,0.6,2504.8,520.730479


There does not appear to be any impossible values here.

<br>

---



## **4) Territory Table**

Check length of rows and columns - 11 rows and 4 columns. <br>
Check all variables are correct data type - they are.

**a) Check for any null, duplicated or erroneous values**

In [39]:
query = '''
SELECT *
FROM `beecycle-413111.BeeCycle.dim_territory`
'''

df = gcpdf(query)
df

,territory_id,region,country,groups
0,11,None,None,None
1,7,France,France,Europe
2,8,Germany,Germany,Europe
3,10,United Kingdom,United Kingdom,Europe
4,9,Australia,Australia,Pacific
5,1,Northwest,United States,North America
6,2,Northeast,United States,North America
7,3,Central,United States,North America
8,4,Southwest,United States,North America
9,5,Southeast,United States,North America


There are no duplicates or erroneous information. However, we can see that territory id 11 is a null entry. We're going to check whether territory id 11 appears in the sales dataset:

In [40]:
query = '''
SELECT DISTINCT territory_id
FROM `beecycle-413111.BeeCycle.fact_sales`
ORDER BY territory_id ASC;
'''

df = gcpdf(query)
df

,territory_id
0,1
1,4
2,5
3,6
4,7
5,8
6,9
7,10


It doesn't so we can delete it from our dataset:

In [ ]:
query = '''
DELETE FROM `beecycle-413111.BeeCycle.dim_territory`
WHERE territory_id = 11;
'''

df = gcpdf(query)
df

<br>

---



## **5) Sales Table**

Check length of rows and columns - 5954 rows and 12 columns. <br>
Check all variables are correct data type - they are.

**a) Preview table**

In [41]:
query = '''
SELECT *
FROM `beecycle-413111.BeeCycle.fact_sales`
LIMIT 3;
'''

df = gcpdf(query)
df

,order_detail_id,order_date,product_id,customer_id,territory_id,sales_order_number,sales_order_line_number,quantity,unitprice,totalprice,totalcost,shippingprice
0,SO63339-1,2020-02-01,353,15491,5,SO63339,1,1,1623.99,1623.993,885.93,40.59986
1,SO63339-3,2020-02-01,473,15491,5,SO63339,3,1,44.45,44.450,16.62,1.11125
2,SO63339-4,2020-02-01,477,15491,5,SO63339,4,1,3.49,3.493,1.31,0.08736


**b) Check for any null values**

In [42]:
query = '''
SELECT *
FROM `beecycle-413111.BeeCycle.fact_sales`
WHERE order_detail_id IS NULL OR order_date IS NULL OR product_id IS NULL OR customer_id IS NULL OR territory_id IS NULL OR sales_order_number IS NULL OR sales_order_line_number IS NULL OR unitprice IS NULL OR totalprice IS NULL OR totalcost IS NULL OR shippingprice IS NULL;
'''

df = gcpdf(query)
df

,order_detail_id,order_date,product_id,customer_id,territory_id,sales_order_number,sales_order_line_number,quantity,unitprice,totalprice,totalcost,shippingprice
0,SO9999-1,2022-06-05,351,<NA>,9,SO9999,1,1,2362.49,2362.493,1328.67,59.06235


There is one entry that has a missing customer id. However, we will keep this row in the dataset as it still contains important information that can be used in analyses.

**c) Check for any duplicate values**

In [43]:
query = '''
SELECT COUNT(*)
FROM `beecycle-413111.BeeCycle.fact_sales`
GROUP BY order_detail_id, order_date, product_id, customer_id, territory_id, sales_order_number,  sales_order_line_number, unitprice, totalprice, totalcost, shippingprice
HAVING COUNT(*) > 1;
'''

df = gcpdf(query)
df

,f0_


There are no duplicates.

**d) Check how many unique values there are for certain variables**

In [44]:
query = '''
SELECT
  COUNT(DISTINCT order_detail_id) AS id_count,
  COUNT(DISTINCT product_id) AS product_count,
  COUNT(DISTINCT customer_id) AS customer_count,
  COUNT(DISTINCT territory_id) AS territory_count,
  COUNT(DISTINCT sales_order_number) AS order_number_count
FROM `beecycle-413111.BeeCycle.fact_sales`;
'''

df = gcpdf(query)
df

,id_count,product_count,customer_count,territory_count,order_number_count
0,5954,148,1357,8,2860


There are 5954 unique order id's so 5954 transactions. 148 different products have been bought in 8 different regions. There are 2860 different order numbers. <br>
1357 customers have made a purchase - this corresponds with what we found in the customers table where 3 out of 1360 customers hadn't made a purchase yet. <br>

**e) Check the minimum and maximum of order date and quantity**

In [45]:
query = '''
SELECT
  MIN(order_date) AS min_order_date,
  MAX(order_date) AS max_order_date,
  MIN(quantity) AS min_quantity,
  MAX(quantity) AS max_quantity
FROM `beecycle-413111.BeeCycle.fact_sales`;
'''

df = gcpdf(query)
df

,min_order_date,max_order_date,min_quantity,max_quantity
0,2017-07-01,2022-06-05,1,1


The minimum and maximum order dates are both possible values. The output also shows that customers only ever by one quantity of something at a time.

**f) Check the pricing variables**

In [46]:
query = '''
SELECT
  MIN(unitprice) AS min_unitprice,
  MAX(unitprice) AS max_unitprice,
  AVG(unitprice) AS average_unitprice,
  MIN(totalprice) AS min_totalprice,
  MAX(totalprice) AS max_totalprice,
  AVG(totalprice) AS average_totalprice,
  MIN(totalcost) AS min_totalcost,
  MAX(totalcost) AS max_totalcost,
  AVG(totalcost) AS average_totalcost,
  MIN(shippingprice) AS min_shippingprice,
  MAX(shippingprice) AS max_shippingprice,
  AVG(shippingprice) AS average_shippingprice
FROM `beecycle-413111.BeeCycle.fact_sales`;
'''

df = gcpdf(query)
df

,min_unitprice,max_unitprice,average_unitprice,min_totalprice,max_totalprice,average_totalprice,min_totalcost,max_totalcost,average_totalcost,min_shippingprice,max_shippingprice,average_shippingprice
0,1.6,2504.79,648.797491,1.603,2504.789,648.799169,0.6,1519.91,376.44307,0.04011,62.61976,16.220011


There does appear to be any out of range values here.

**g) Perform a couple of calculations**

In [47]:
query = '''
SELECT
  SUM(totalprice) AS total_revenue,
  SUM(totalprice - totalcost) AS total_profit
FROM `beecycle-413111.BeeCycle.fact_sales`;
'''

df = gcpdf(query)
df

,total_revenue,total_profit
0,3.862950e+06,1.621608e+06


The total revenue is £3,862,950. The total profits are £1,621,608.